In [3]:
import pandas as pd
import os

import win32com.client as win32


ModuleNotFoundError: No module named 'win32com'

In [6]:
# Load
base_dir = os.path.abspath(os.path.join('..', '..', 'db_softtek'))
caminho_cubo_anl = os.path.join(base_dir, 'cubo_anl')
caminho_modelo = os.path.join(base_dir, 'modelo')

csv_file_name = 'novos_chamados.csv'
csv_path = os.path.join(caminho_modelo, csv_file_name)

novos_chamados_df = pd.read_csv(csv_path)


In [7]:

# Group by the 'IS' column and aggregate the number and type of calls
Chamados_por_consultor = novos_chamados_df.groupby(['IS']).agg({
    'CHAMADO': 'count',
    'TIPO_CHAMADO': lambda x: '|'.join(set(x))
}).reset_index()

# Create an instance of Outlook
outlook = win32.Dispatch('outlook.application')

# Iterate over each consultant in the grouped DataFrame
for index, row in Chamados_por_consultor.iterrows():
    funcionario = row['IS']
    qtd_chamados = row['CHAMADO']
    tipos_chamados = row['TIPO_CHAMADO']

    try:
        # Create a new email
        mail = outlook.CreateItem(0)
        
        # Set the recipient email (adjust as needed)
        mail.To = f'{funcionario}@softtek.com.br'

        # Set the subject of the email
        mail.Subject = f'Atenção! Você está com {qtd_chamados} chamados para tratamento!'

        # Set the body of the email
        mail.Body = f"""
        Olá {funcionario},

        Você tem {qtd_chamados} chamado(s) pendente(s). Os tipos de chamados são os seguintes:

        {tipos_chamados.replace('|', '\n- ')}

        Por favor, verifique e tome as medidas necessárias.

        Atenciosamente,
        Gestão
        """

        # Send the email
        mail.Send()

    except Exception as e:
        print(f'Falha ao enviar e-mail para {funcionario}@softtek.com.br Erro: {str(e)}')

print('E-mails enviados com sucesso!')

NameError: name 'win32' is not defined